## Imports

In [1]:
import pandas as pd
import numpy as np

## Import Data

In [25]:
titles_imdb = pd.read_csv('./../data/imdb_movie_info.tsv', sep='\t')
ratings_imdb = pd.read_csv('./../data/imdb_movie_ratings.tsv', sep='\t')
budgets_kaggle = pd.read_csv('./../data/kaggle_movies_metadata.csv')

/var/folders/g4/ttsfd72s5qn1w8w3x92b3yyw0000gn/T/ipykernel_49376/183344427.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles_imdb = pd.read_csv('./../data/imdb_movie_info.tsv', sep='\t')
/var/folders/g4/ttsfd72s5qn1w8w3x92b3yyw0000gn/T/ipykernel_49376/183344427.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  budgets_kaggle = pd.read_csv('./../data/kaggle_movies_metadata.csv')


In [3]:
titles_imdb.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
ratings_imdb.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1922
1,tt0000002,5.8,259
2,tt0000003,6.5,1734
3,tt0000004,5.6,174
4,tt0000005,6.2,2545


In [26]:
budgets_kaggle.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [83]:
# Importing CMU Datasets
movie = pd.read_csv('./../data/MovieSummaries/movie.metadata.tsv', 
                 sep='\t',
                 names=['Wikipedia_movie_ID',
                        'Freebase_movie_ID',
                        'Movie_name',
                        'Movie_release_date',
                        'Movie_box_office_revenue',
                        'Movie_runtime',
                        'Movie_languages',
                        'Movie_countries',
                        'Movie_genres'],
                 header=None)

character = pd.read_csv('./../data/MovieSummaries/character.metadata.tsv', 
                 sep='\t',
                 names=['Wikipedia_movie_ID',
                        'Freebase_movie_ID',
                        'Movie_release_date',
                        'Character_name',
                        'Actor_date_of_birth',
                        'Actor_gender',
                        'Actor_height',
                        'Actor_ethnicity',
                        'Actor_name',
                        'Actor_age_at_movie_release',
                        'Freebase_character/actor_map_ID',
                        'Freebase_character_ID',
                        'Freebase_actor_ID'],
                 header=None)

In [6]:
movie.head()

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [7]:
character.head()

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_release_date,Character_name,Actor_date_of_birth,Actor_gender,Actor_height,Actor_ethnicity,Actor_name,Actor_age_at_movie_release,Freebase_character/actor_map_ID,Freebase_character_ID,Freebase_actor_ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


# Data cleaning

### Date change

In [8]:
#move release dates and actor birth dates to datetime format and keep only release year
character.Movie_release_date = pd.to_datetime(character['Movie_release_date'],
                                              errors='coerce').dt.year

movie.Movie_release_date = pd.to_datetime(movie['Movie_release_date'],
                                              errors='coerce').dt.year

In [9]:
#sort dataframes by ascending release year
character.sort_values(by=['Movie_release_date'], ascending=True, inplace=True)

movie.sort_values(by=['Movie_release_date'], ascending=True, inplace=True)

### Null value removal

In [10]:
# Nan percentage for character dataset
character.isnull().sum() * 100 / character.shape[0]

Wikipedia_movie_ID                  0.000000
Freebase_movie_ID                   0.000000
Movie_release_date                  2.221142
Character_name                     57.220488
Actor_date_of_birth                23.552763
Actor_gender                       10.120288
Actor_height                       65.645740
Actor_ethnicity                    76.466542
Actor_name                          0.272484
Actor_age_at_movie_release         35.084064
Freebase_character/actor_map_ID     0.000000
Freebase_character_ID              57.218269
Freebase_actor_ID                   0.180842
dtype: float64

In [84]:
character = character[['Wikipedia_movie_ID', 'Actor_name', 'Actor_gender', 'Actor_date_of_birth']]
character

,Wikipedia_movie_ID,Actor_name,Actor_gender,Actor_date_of_birth
0,975900,Wanda De Jesus,F,1958-08-26
1,975900,Natasha Henstridge,F,1974-08-15
2,975900,Ice Cube,M,1969-06-15
3,975900,Jason Statham,M,1967-09-12
4,975900,Clea DuVall,F,1977-09-25
...,...,...,...,...
450664,913762,Dorothy Elias-Fahn,F,1970-05
450665,913762,Jonathan Fahn,M,1965-04-12
450666,28308153,David Hemmings,M,1941-11-18
450667,28308153,Roberta Paterson,NaN,NaN


In [85]:
char_no_nan = character.dropna(axis=0, how='any')
char_no_nan

,Wikipedia_movie_ID,Actor_name,Actor_gender,Actor_date_of_birth
0,975900,Wanda De Jesus,F,1958-08-26
1,975900,Natasha Henstridge,F,1974-08-15
2,975900,Ice Cube,M,1969-06-15
3,975900,Jason Statham,M,1967-09-12
4,975900,Clea DuVall,F,1977-09-25
...,...,...,...,...
450661,913762,Sonny Byrkett,M,1954
450662,913762,Susan Byrkett,F,1958
450664,913762,Dorothy Elias-Fahn,F,1970-05
450665,913762,Jonathan Fahn,M,1965-04-12


In [39]:
# Nan percentage for movie dataset
movie.isnull().sum() * 100 / movie.shape[0]

Movie_name                   0.000000
Movie_release_date           8.444966
Movie_box_office_revenue    89.722416
dtype: float64

In [86]:
movie = movie[['Wikipedia_movie_ID', 'Movie_name', 'Movie_release_date', 'Movie_box_office_revenue']]
movie

,Wikipedia_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue
0,975900,Ghosts of Mars,2001-08-24,14010832.0
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN
2,28463795,Brun bitter,1988,NaN
3,9363483,White Of The Eye,1987,NaN
4,261236,A Woman in Flames,1983,NaN
...,...,...,...,...
81736,35228177,Mermaids: The Body Found,2011-03-19,NaN
81737,34980460,Knuckle,2011-01-21,NaN
81738,9971909,Another Nice Mess,1972-09-22,NaN
81739,913762,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN


In [87]:
movie_no_nan = movie.dropna(0, inplace= False, how='any')
movie_no_nan

/var/folders/g4/ttsfd72s5qn1w8w3x92b3yyw0000gn/T/ipykernel_49376/2940552793.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  movie_no_nan = movie.dropna(0, inplace= False, how='any')


,Wikipedia_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue
0,975900,Ghosts of Mars,2001-08-24,14010832.0
7,10408933,Alexander's Ragtime Band,1938-08-16,3600000.0
13,171005,Henry V,1989-11-08,10161099.0
17,77856,Mary Poppins,1964-08-27,102272727.0
21,612710,New Rose Hotel,1999-10-01,21521.0
...,...,...,...,...
81695,54540,Coming to America,1988-06-29,288752301.0
81720,7761830,Spaced Invaders,1990,15369573.0
81725,1918494,State and Main,2000-08-26,6944471.0
81726,664006,Guilty as Sin,1993-06-04,22886222.0


In [57]:
budgets_kaggle = budgets_kaggle[['original_title', 'budget']]
budgets_kaggle

,original_title,budget
0,Toy Story,30000000
1,Jumanji,65000000
2,Grumpier Old Men,0
3,Waiting to Exhale,16000000
4,Father of the Bride Part II,0
...,...,...
45461,رگ خواب,0
45462,Siglo ng Pagluluwal,0
45463,Betrayal,0
45464,Satana likuyushchiy,0


In [58]:
budgets_kaggle['budget'] = budgets_kaggle['budget'].replace('0', np.nan)
budgets_kaggle.isna().sum()

original_title        0
budget            36573
dtype: int64

In [68]:
budget_no_nan = budgets_kaggle.dropna()

### Adjusting for Inflation

# Merging Datasets

In [42]:
# Merging movie titles dataset with ratings dataset
first_merge = titles_imdb.merge(ratings_imdb, how='inner')
first_merge.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.7,1922
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",5.8,259
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.5,1734
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",5.6,174
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.2,2545


In [43]:
# Droping unnecessary columns
imdb = first_merge[['primaryTitle', 'averageRating', 'numVotes']]
imdb.head()

,primaryTitle,averageRating,numVotes
0,Carmencita,5.7,1922
1,Le clown et ses chiens,5.8,259
2,Pauvre Pierrot,6.5,1734
3,Un bon bock,5.6,174
4,Blacksmith Scene,6.2,2545


In [44]:
# Removing duplicate film titles and averaging the duplicate ratings
mean_rating_imdb = first_merge[['primaryTitle', 'averageRating']].groupby('primaryTitle').mean()
mean_rating_imdb

,averageRating
primaryTitle,
!Next?,5.20
!Que ve el Bisbe!,6.20
!Women Art Revolution,6.80
#,4.25
# My Ass,6.80
...,...
ärtico,5.60
él,6.10
êmîcêtôsêt: Many Bloodlines,8.30


In [45]:
# Removing duplicate film titles and adding together the duplicate votes
total_vote_imdb = first_merge[['primaryTitle', 'numVotes']].groupby('primaryTitle').sum()
total_vote_imdb

,numVotes
primaryTitle,
!Next?,18
!Que ve el Bisbe!,12
!Women Art Revolution,247
#,16
# My Ass,9
...,...
ärtico,120
él,19
êmîcêtôsêt: Many Bloodlines,14


In [46]:
# Merging the two above dataframes
no_dupe_imdb = mean_rating_imdb.merge(total_vote_imdb, 'inner', 'primaryTitle')
no_dupe_imdb

,averageRating,numVotes
primaryTitle,,
!Next?,5.20,18
!Que ve el Bisbe!,6.20,12
!Women Art Revolution,6.80,247
#,4.25,16
# My Ass,6.80,9
...,...,...
ärtico,5.60,120
él,6.10,19
êmîcêtôsêt: Many Bloodlines,8.30,14


In [88]:
# Merging the rating/votes dataframe with movie metadata with no nan values
rated_data = movie_no_nan.merge(no_dupe_imdb, 'inner', left_on='Movie_name', right_on='primaryTitle')
rated_data

,Wikipedia_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,averageRating,numVotes
0,975900,Ghosts of Mars,2001-08-24,14010832.0,6.400000,55259
1,10408933,Alexander's Ragtime Band,1938-08-16,3600000.0,5.550000,2181
2,171005,Henry V,1989-11-08,10161099.0,6.811111,38150
3,77856,Mary Poppins,1964-08-27,102272727.0,7.816667,173430
4,612710,New Rose Hotel,1999-10-01,21521.0,5.100000,5541
...,...,...,...,...,...,...
7764,1191380,Wilde,1997,2158775.0,6.900000,17320
7765,54540,Coming to America,1988-06-29,288752301.0,7.233333,208953
7766,7761830,Spaced Invaders,1990,15369573.0,6.900000,3758
7767,1918494,State and Main,2000-08-26,6944471.0,6.700000,21485


In [90]:
data = pd.merge(char_no_nan, rated_data)
data = data.drop(['Wikipedia_movie_ID'], axis=1)
data

,Actor_name,Actor_gender,Actor_date_of_birth,Movie_name,Movie_release_date,Movie_box_office_revenue,averageRating,numVotes
0,Wanda De Jesus,F,1958-08-26,Ghosts of Mars,2001-08-24,14010832.0,6.400000,55259
1,Natasha Henstridge,F,1974-08-15,Ghosts of Mars,2001-08-24,14010832.0,6.400000,55259
2,Ice Cube,M,1969-06-15,Ghosts of Mars,2001-08-24,14010832.0,6.400000,55259
3,Jason Statham,M,1967-09-12,Ghosts of Mars,2001-08-24,14010832.0,6.400000,55259
4,Clea DuVall,F,1977-09-25,Ghosts of Mars,2001-08-24,14010832.0,6.400000,55259
...,...,...,...,...,...,...,...,...
79046,Roland Culver,M,1900-08-31,The Yellow Rolls-Royce,1964-12-31,5400000.0,6.400000,3656
79047,Michael Hordern,M,1911-10-03,The Yellow Rolls-Royce,1964-12-31,5400000.0,6.400000,3656
79048,Reginald Beckwith,M,1908-11-02,The Yellow Rolls-Royce,1964-12-31,5400000.0,6.400000,3656
79049,Gong Yoo,M,1979-07-10,The Crucible,2011-09-22,30723856.0,7.441667,41823


In [69]:
# Renaming column to avoid merge issues
budget_no_nan = budget_no_nan.rename(columns={'original_title': 'Movie_name'})
budget_no_nan

,Movie_name,budget
0,Toy Story,30000000
1,Jumanji,65000000
3,Waiting to Exhale,16000000
5,Heat,60000000
6,Sabrina,58000000
...,...,...
45402,Корпоратив,2000000
45408,Марс,2000000
45409,Dikari,800000
45412,Про любоff,2000000


In [91]:
# Merging dataset with budget information and the ratings dataset
budget = pd.merge(budget_no_nan, rated_data)
budget

,Movie_name,budget,Wikipedia_movie_ID,Movie_release_date,Movie_box_office_revenue,averageRating,numVotes
0,Toy Story,30000000,53085,1995-11-19,361958736.0,7.781818,990319
1,Jumanji,65000000,3700174,1995-12-15,262797249.0,6.360000,347988
2,Waiting to Exhale,16000000,972970,1995-12-22,81452156.0,7.600000,16224
3,Heat,60000000,43566,1995-12-15,187436818.0,7.202632,659457
4,Heat,60000000,21917672,2006-12-28,15710000.0,7.202632,659457
...,...,...,...,...,...,...,...
4039,The Assignment,5000000,9078818,1997-09-12,332597.0,6.710000,20083
4040,Eating Out,50000,4447058,2004,155212.0,6.640000,6431
4041,I Was a Teenage Werewolf,82000,1180124,1957-06-19,2000000.0,6.350000,3117
4042,The Magic Roundabout,20000000,2349209,2005-02-02,26691243.0,6.533333,474


In [77]:
# Converting the budget column from string to numerical value
budget['budget'] = pd.to_numeric(budget['budget'])

In [79]:
# Creating a Profit column
budget['profit'] = budget['Movie_box_office_revenue'] - budget['budget']
budget

,Movie_name,budget,Movie_release_date,Movie_box_office_revenue,averageRating,numVotes,profit
0,Toy Story,30000000,1995.0,361958736.0,7.781818,990319,331958736.0
1,Jumanji,65000000,1995.0,262797249.0,6.360000,347988,197797249.0
2,Waiting to Exhale,16000000,1995.0,81452156.0,7.600000,16224,65452156.0
3,Heat,60000000,1986.0,2793214.0,7.202632,659457,-57206786.0
4,Heat,60000000,1995.0,187436818.0,7.202632,659457,127436818.0
...,...,...,...,...,...,...,...
4039,The Assignment,5000000,1997.0,332597.0,6.710000,20083,-4667403.0
4040,Eating Out,50000,2004.0,155212.0,6.640000,6431,105212.0
4041,I Was a Teenage Werewolf,82000,1957.0,2000000.0,6.350000,3117,1918000.0
4042,The Magic Roundabout,20000000,2005.0,26691243.0,6.533333,474,6691243.0


In [92]:
# Merging the profit dataset with character dataset
profit = pd.merge(char_no_nan, budget)
profit = profit.drop(['Wikipedia_movie_ID'], axis=1)
profit

,Actor_name,Actor_gender,Actor_date_of_birth,Movie_name,budget,Movie_release_date,Movie_box_office_revenue,averageRating,numVotes
0,Wanda De Jesus,F,1958-08-26,Ghosts of Mars,28000000,2001-08-24,14010832.0,6.400000,55259
1,Natasha Henstridge,F,1974-08-15,Ghosts of Mars,28000000,2001-08-24,14010832.0,6.400000,55259
2,Ice Cube,M,1969-06-15,Ghosts of Mars,28000000,2001-08-24,14010832.0,6.400000,55259
3,Jason Statham,M,1967-09-12,Ghosts of Mars,28000000,2001-08-24,14010832.0,6.400000,55259
4,Clea DuVall,F,1977-09-25,Ghosts of Mars,28000000,2001-08-24,14010832.0,6.400000,55259
...,...,...,...,...,...,...,...,...,...
49263,Russell Peters,M,1970-09-29,Source Code,32000000,2011-03-11,147332697.0,8.200000,520701
49264,James A. Woods,M,1979-10-30,Source Code,32000000,2011-03-11,147332697.0,8.200000,520701
49265,Scott Bakula,M,1954-10-09,Source Code,32000000,2011-03-11,147332697.0,8.200000,520701
49266,Gong Yoo,M,1979-07-10,The Crucible,25000000,2011-09-22,30723856.0,7.441667,41823


In [93]:
profit.to_csv('profit_data.csv')
data.to_csv('rating_no_budget_data.csv')